In [1]:
import sys
import random
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset

In [2]:
# Next-step: one-hot without context and IDs

In [3]:
sys.path.append("..")

In [4]:
from carca.data import CARCADataset, load_attrs, load_ctx, load_profiles, set_datapath
from carca.model import CARCA
from carca.train import train, evaluate

In [5]:
set_datapath("../../data/embedding_experiment/video_games/")

In [6]:
attrs = load_attrs("video_games_sbert_5core.dat")
ctx = load_ctx("video_games_ctx_5core.dat")
user_ids, item_ids, profiles = load_profiles("video_games_sorted_5core.txt")

In [7]:
exp_name = "embedding_experiment_onehot"
n_items = attrs.shape[0]
n_ctx = next(iter(ctx.values())).shape[0]
n_attrs = attrs.shape[1]
embeddings = "attr"

In [8]:
# Hyper-parameters
learning_rate =  0.0001
seq_len = 50
n_blocks = 3
n_heads = 3
dropout_rate = 0.5
l2_reg = 0.0
d_dim = 90
g_dim = 450
residual_sa = True
residual_ca = True
epochs = 800
batch_size = 128
beta1 = 0.9
beta2 = 0.98

In [9]:
train_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="train"
)
val_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="val"
)
test_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="test"
)

val_idx = random.sample(range(len(val_data)), 10_000) if len(val_data) > 10_000 else range(len(val_data))
val_sub = Subset(val_data, val_idx)
test_idx = random.sample(range(len(test_data)), 10_000) if len(test_data) > 10_000 else range(len(test_data))
test_sub = Subset(test_data, test_idx)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_sub, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_sub, batch_size=batch_size, shuffle=False)

In [10]:
model = CARCA(
    n_items=n_items,
    d=d_dim,
    g=g_dim,
    n_ctx=n_ctx,
    n_attrs=n_attrs,
    H=n_heads,
    p=dropout_rate,
    B=n_blocks,
    res_sa=residual_sa,
    res_ca=residual_ca,
    embeddings=embeddings
)

In [11]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda"
print(f"Using {device} device")
model = model.to(device)

Using cuda device


In [12]:
optim = Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg, betas=(beta1, beta2))

In [13]:
model = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    optim=optim,
    epochs=epochs,
    early_stop=50,
    checkpoint=f"./{exp_name}"
    # scheduler=scheduler
)

20:48:55 - Epoch 001: Loss = 0.5977
20:49:02 - Epoch 001: Loss = 0.5378 HR = 0.5282, NDCG = 0.3168
20:49:34 - Epoch 002: Loss = 0.4783
20:49:40 - Epoch 002: Loss = 0.5474 HR = 0.5738, NDCG = 0.3376
20:50:10 - Epoch 003: Loss = 0.4558
20:50:16 - Epoch 003: Loss = 0.5055 HR = 0.5602, NDCG = 0.3325
20:50:44 - Epoch 004: Loss = 0.4392
20:50:50 - Epoch 004: Loss = 0.4968 HR = 0.5751, NDCG = 0.3417
20:51:18 - Epoch 005: Loss = 0.4279
20:51:24 - Epoch 005: Loss = 0.4941 HR = 0.5826, NDCG = 0.3461
20:51:52 - Epoch 006: Loss = 0.4191
20:51:58 - Epoch 006: Loss = 0.4896 HR = 0.5849, NDCG = 0.3511
20:52:26 - Epoch 007: Loss = 0.4131
20:52:31 - Epoch 007: Loss = 0.4929 HR = 0.5964, NDCG = 0.3585
20:53:00 - Epoch 008: Loss = 0.4077
20:53:05 - Epoch 008: Loss = 0.4870 HR = 0.5930, NDCG = 0.3536
20:53:33 - Epoch 009: Loss = 0.4029
20:53:39 - Epoch 009: Loss = 0.4826 HR = 0.6040, NDCG = 0.3634
20:54:07 - Epoch 010: Loss = 0.3955
20:54:13 - Epoch 010: Loss = 0.4643 HR = 0.6096, NDCG = 0.3671
20:54:41 -

KeyboardInterrupt: 